In [1]:
import os
import webdataset as wds
import logging
import numpy
import json
import torch
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from pathlib import Path
from typing import Optional
from itertools import islice

from torch.utils.data import DataLoader, IterableDataset
from webdataset.filters import pipelinefilter
from webdataset.utils import pytorch_worker_info


In [2]:
!ls YandexCup2024v2/YaCupTrain/0/
!ls YandexCup2024v2/YaCupTest/0/


control.csv			  localization-quant20mc_v3.csv
control_norm_v1.csv		  localization-quant40mc_v4.csv
control_norm_v1-quant20mc.csv	  localization-quant80mc_v5.csv
control_norm_v1-quant20mc_v2.csv  metadata.json
control_norm_v1-quant20mc_v3.csv  quant20mc
control_norm_v1-quant40mc_v4.csv  quant20mc_v3
control_norm_v1-quant80mc_v5.csv  quant40mc_v4
localization.csv		  quant80mc_v5
localization-quant20mc.csv	  speed-norm-quant20mc_v2.csv
localization-quant20mc_v2.csv	  speed-quant20mc_v2.csv
control.csv			  metadata.json
control_norm_v1.csv		  quant20mc
control_norm_v1-quant20mc.csv	  quant20mc_v3
control_norm_v1-quant20mc_v2.csv  quant40mc_v4
control_norm_v1-quant20mc_v3.csv  quant80mc_v5
control_norm_v1-quant40mc_v4.csv  requested_stamps.csv
control_norm_v1-quant80mc_v5.csv  requested_stamps-quant20mc.csv
localization.csv		  requested_stamps-quant20mc_v2.csv
localization-quant20mc.csv	  requested_stamps-quant20mc_v3.csv
localization-quant20mc_v2.csv	  requested_stamps-quant20mc_v4.csv
loc

In [3]:
!head YandexCup2024v2/YaCupTrain/0/control_norm_v1-quant20mc_v3.csv

stamp_ns,acceleration_level,steering,norm_acceleration_level,norm_steering
0.0,-114.0,-2.655139533023165,-0.4514997100029219,-0.0615109367890647
1.0,-118.51124043222295,-2.6265831542518203,-0.4812146939425406,-0.06101266412300235
2.0,-123.02273633437042,-2.5980333021733926,-0.5109313606310214,-0.060514505339470656
3.0,-127.58524197730276,-2.57078663642703,-0.5409840221863152,-0.06003908550800475
4.0,-132.14585451381703,-2.5444222994575325,-0.5710242140832847,-0.0595790611969412
5.0,-136.64990027292015,-2.5444222994575325,-0.6006918075990458,-0.0595790611969412
6.0,-141.10246811465205,-2.5434682242216784,-0.6300203224266029,-0.05956241379163819
7.0,-144.10040882917525,-2.5155545556234253,-0.6497673879005857,-0.05907535560462499
8.0,-147.08219304041114,-2.487715056006009,-0.6694080324814873,-0.05858959157245253


In [4]:
!head YandexCup2024v2/YaCupTrain/0/localization-quant20mc_v3.csv

stamp_ns,x,y,z,roll,pitch,yaw
-149.0,-4292.346493792404,-14527.351865581526,66.04828378656846,0.0038841648135669667,-0.054192977118628395,-1.9368185091788004
-148.0,-4292.434627804271,-14527.581805518565,66.06164121751011,0.0037722121488635474,-0.0541798604632994,-1.9368425656572617
-147.0,-4292.522253342697,-14527.81156859566,66.07382280575607,0.0034592108718404678,-0.05425698778655318,-1.9368518636282488
-146.0,-4292.609022484208,-14528.041033796213,66.08402397974696,0.0028075932652278372,-0.054486108622434,-1.9368363045680264
-145.0,-4292.696232114875,-14528.270457198889,66.09546114109773,0.002455461353684493,-0.05461626036609381,-1.9368380534109395
-144.0,-4292.7841572669495,-14528.499812705968,66.1089060145706,0.0025898077670152724,-0.054585648777846066,-1.9368679168511878
-143.0,-4292.871967887383,-14528.729133961555,66.12196841585748,0.0027456198291566087,-0.05449949224467743,-1.9368931484338072
-142.0,-4292.958891041339,-14528.958189814079,66.13206717112308,0.003067761854690518

In [5]:
!head YandexCup2024v2/YaCupTrain/0/metadata.json

{"ride_date": "2022-03-14", "tires": {"front": 0, "rear": 0}, "vehicle_id": 0, "vehicle_model": 0, "vehicle_model_modification": 0, "location_reference_point_id": 0}

In [16]:
os.listdir('YandexCup2024v2/YaCupTest/0')

['metadata.json',
 'control.csv',
 'localization.csv',
 'requested_stamps.csv',
 'control_norm_v1.csv',
 'control_norm_v1-quant20mc.csv',
 'localization-quant20mc.csv',
 'requested_stamps-quant20mc.csv',
 'quant20mc',
 'control_norm_v1-quant20mc_v2.csv',
 'localization-quant20mc_v2.csv',
 'requested_stamps-quant20mc_v2.csv',
 'speed-quant20mc_v2.csv',
 'speed-norm-quant20mc_v2.csv',
 'control_norm_v1-quant20mc_v3.csv',
 'localization-quant20mc_v3.csv',
 'requested_stamps-quant20mc_v3.csv',
 'quant20mc_v3',
 'control_norm_v1-quant40mc_v4.csv',
 'localization-quant40mc_v4.csv',
 'requested_stamps-quant20mc_v4.csv',
 'quant40mc_v4',
 'control_norm_v1-quant80mc_v5.csv',
 'localization-quant80mc_v5.csv',
 'requested_stamps-quant20mc_v5.csv',
 'quant80mc_v5']

In [6]:
def configure_logging(log_level='INFO'):
    handlers =  {
            "base_out": {
                "class": "logging.StreamHandler",
                "formatter": "basic",
                "stream": "ext://sys.stderr",
            }
    }
    CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {"basic": {"format": '%(asctime)s %(name)s %(pathname)s:%(lineno)d - %(levelname)s - %(message)s'}},
        "handlers": handlers,
        "loggers": {"base": {"handlers": handlers.keys(), "level": log_level}},
        "root": {"handlers": handlers.keys(), "level": log_level}
    }
    logging.config.dictConfig(CONFIG)
configure_logging()


In [7]:
class DirLoader(IterableDataset):
    def __init__(self, root_dir, 
                 control_bname,
                loc_bname,
                 start_bname,
                req_bname=None,
                meta_bname='metadata.json',
                seed=None,
                pad_value= -1000000):
        super().__init__()
        self.root_dir = root_dir
        self.control_bname = control_bname
        self.loc_bname = loc_bname
        self.req_bname = req_bname
        self.meta_bname = meta_bname
        self.start_bname = start_bname
        self.seed = seed
        self.pad_value = pad_value
        self.dirlist = [*os.listdir(root_dir)]

    def __len__(self):
        return len(self.dirlist)

    def __iter__(self):
        dirlist = self.dirlist
        rank, world_size, worker, num_workers = pytorch_worker_info()
        assert world_size == 1, "Do not use this class for DDP"
        if num_workers > 1:
            full_len = len(dirlist)
            dirlist = list(islice(dirlist, worker, None, num_workers))
            logging.info(
                f"Subset for {worker} worker contains {len(dirlist)}/{full_len} annotations"
            )
            logging.debug(f"First dir is {dirlist[0]}")
        if len(dirlist) == 0:
            logging.warning(
                f"Zero len dirs list! {worker=}, {num_workers=}, {len(dirlist)=}, {len(self.dirlist)}"
            )
            return

        if self.seed is not None:
            random.Random(self.seed).shuffle(dirlist)
        for dbname in dirlist:
            d = f'{self.root_dir}/{dbname}'
            control_df = pd.read_csv(f"{d}/{self.control_bname}")
            loc_df = pd.read_csv(f"{d}/{self.loc_bname}")
            df = control_df.set_index('stamp_ns').join(loc_df.set_index('stamp_ns'))
            df = df.fillna(self.pad_value)
            with open(f"{d}/{self.meta_bname}") as f:
                meta = json.load(f)
            assert np.allclose(df.index, np.arange(len(df))), f"{len(df)=}, \n{df.stamp_ns=}"
            out = {
                '__key__': f"{Path(self.root_dir).stem}__{Path(d).stem}",
                'frame_id.pth': torch.from_numpy(df.index.values),
                'c_stamps.pth': torch.from_numpy(control_df.stamp_ns.values),
                'acceleration_level.pth': torch.from_numpy(df.norm_acceleration_level.values),
                'steering.pth' : torch.from_numpy(df.norm_steering.values),
                'l_stamps.pth': torch.from_numpy(loc_df.stamp_ns.values),
                'x.pth': torch.from_numpy(df.x.values),
                'y.pth': torch.from_numpy(df.y.values),
                'z.pth': torch.from_numpy(df.z.values),
                'roll.pth': torch.from_numpy(df.roll.values),
                'pitch.pth': torch.from_numpy(df.pitch.values),
                'yaw.pth': torch.from_numpy(df.yaw.values),
                'ride_date.txt': meta['ride_date'],
                "tires.pickle": meta['tires'],
                "vehicle_id.id": meta['vehicle_id'], 
                "vehicle_model.id": meta['vehicle_model'], 
                "vehicle_model_modification.id": meta['vehicle_model_modification'], 
                "location_reference_point_id.id": meta['location_reference_point_id']
            }
            if self.req_bname is not None:
                req_df = pd.read_csv(f"{d}/{self.req_bname}")
                out['req_stamps.pth'] = torch.from_numpy(req_df.stamp_ns.values)
            if self.start_bname is not None:
                with open(f"{d}/{self.start_bname}") as f:
                    start_ns = int(float(f.read()))
                out['start_ns.id'] = start_ns
            yield out

In [8]:
def _prepare_input_feats_v1(data):
    for e in data:
        T = e['frame_id.pth'].shape[0]
        al = e.pop('acceleration_level.pth')
        s = e.pop('steering.pth')
        # 4 X T
        e['control_feats.pth'] = torch.stack([al, 
                                    s, 
                                    torch.full((T,), e['vehicle_model.id']),
                                    torch.full((T,), e['vehicle_model_modification.id'])
                                    ])
        x = e.pop('x.pth')
        y = e.pop('y.pth')
        z = e.pop('z.pth')
        roll = e.pop('roll.pth')
        pitch = e.pop('pitch.pth')
        yaw = e.pop('yaw.pth')
        e['loc.pth'] =  torch.stack([x, 
                               y,
                               z,
                               roll,
                               pitch,
                               yaw])
        yield e
        
prepare_input_feats_v1  = pipelinefilter(_prepare_input_feats_v1)

In [20]:
#!rm -rf exp_v1

In [9]:
for d in ['YandexCup2024v2/YaCupTrain/']:
    out_d = f'exp_v5/{d}'
    os.makedirs(out_d, exist_ok=False)
    max_elements_per_shard=50
    num_e = len(os.listdir(d))
    with wds.ShardWriter(f'{out_d}/dump-%06d.tar', maxcount=400) as sink:
        for e in tqdm(
            _prepare_input_feats_v1(
                DataLoader(
                    DirLoader(d, 
                              control_bname='control_norm_v1-quant80mc_v5.csv',
                              loc_bname='localization-quant80mc_v5.csv',
                              req_bname=None,
                              meta_bname='metadata.json',
                              start_bname='quant80mc_v5',
                              seed=42), 
                    batch_size=None, 
                    num_workers=6,
                    sampler=None
                )
            ), 
            total=num_e
        ):
            sink.write(e)
    print(f'Done {out_d}')

# writing exp_v5/YandexCup2024v2/YaCupTrain//dump-000000.tar 0 0.0 GB 0


  0%|                                                                                                                                                                                                                                                   | 0/42000 [00:00<?, ?it/s]2024-11-03 00:03:41,465 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 0 worker contains 7000/42000 annotations
2024-11-03 00:03:41,495 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 2 worker contains 7000/42000 annotations
2024-11-03 00:03:41,495 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 1 worker contains 7000/42000 annotations
2024-11-03 00:03:41,524 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 3 worker contains 7000/42000 annotations
2024-11-03 00:03:41,528 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 4 worker contains 7000/42000 annotations
2024-11-03 00:03:41,555 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subs

# writing exp_v5/YandexCup2024v2/YaCupTrain//dump-000001.tar 400 0.0 GB 400


  2%|████▍                                                                                                                                                                                                                                    | 809/42000 [00:10<08:20, 82.29it/s]

# writing exp_v5/YandexCup2024v2/YaCupTrain//dump-000002.tar 400 0.0 GB 800


...

# writing exp_v5/YandexCup2024v2/YaCupTrain//dump-000103.tar 400 0.0 GB 41200


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 41621/42000 [06:30<00:02, 128.29it/s]

# writing exp_v5/YandexCup2024v2/YaCupTrain//dump-000104.tar 400 0.0 GB 41600


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42000/42000 [06:33<00:00, 106.74it/s]

Done exp_v5/YandexCup2024v2/YaCupTrain/


In [10]:
for d in ['YandexCup2024v2/YaCupTest/']:
    out_d = f'exp_v5/{d}'
    os.makedirs(out_d, exist_ok=False)
    max_elements_per_shard=50
    num_e = len(os.listdir(d))
    with wds.ShardWriter(f'{out_d}/dump-%06d.tar', maxcount=400) as sink:
        for e in tqdm(
            _prepare_input_feats_v1(
                DataLoader(
                    DirLoader(d, 
                              control_bname='control_norm_v1-quant80mc_v5.csv',
                              loc_bname='localization-quant80mc_v5.csv',
                              meta_bname='metadata.json',
                              start_bname='quant80mc_v5',
                              req_bname='requested_stamps-quant80mc_v5.csv', seed=None), 
                    batch_size=None, 
                    num_workers=6,
                    sampler=None
                )
            ), 
            total=num_e
        ):
            sink.write(e)
    print(f'Done {out_d}')

# writing exp_v5/YandexCup2024v2/YaCupTest//dump-000000.tar 0 0.0 GB 0


  0%|                                                                                                                                                                                                                                                    | 0/8000 [00:00<?, ?it/s]2024-11-03 00:10:15,007 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 0 worker contains 1334/8000 annotations
2024-11-03 00:10:15,011 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 2 worker contains 1333/8000 annotations
2024-11-03 00:10:15,011 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 1 worker contains 1334/8000 annotations
2024-11-03 00:10:15,038 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 4 worker contains 1333/8000 annotations
2024-11-03 00:10:15,038 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset for 3 worker contains 1333/8000 annotations
2024-11-03 00:10:15,042 root /tmp/ipykernel_3803739/965619569.py:31 - INFO - Subset fo

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/mnt/asr_hot/mitrofanov-aa/projects/audiollm/audiollm_stc_recipe/anaconda/envs/pl/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/mnt/asr_hot/mitrofanov-aa/projects/audiollm/audiollm_stc_recipe/anaconda/envs/pl/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 42, in fetch
    data = next(self.dataset_iter)
  File "/tmp/ipykernel_3803739/965619569.py", line 73, in __iter__
    req_df = pd.read_csv(f"{d}/{self.req_bname}")
  File "/mnt/asr_hot/mitrofanov-aa/projects/audiollm/audiollm_stc_recipe/anaconda/envs/pl/lib/python3.9/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/mnt/asr_hot/mitrofanov-aa/projects/audiollm/audiollm_stc_recipe/anaconda/envs/pl/lib/

In [20]:
FRAME_STEP = 20000000

def measure_score(df_pred, loc_df):
    df_join = df_pred.set_index('stamp_ns').join(loc_df[['stamp_ns', 'x', 'y', 'yaw']].set_index('stamp_ns'))
    df_join['mseX'] = ((df_join['pred_x'] - df_join.x)**2 + (df_join['pred_y'] - df_join.y)**2)**(1/2)
    df_join['mse_yaw'] = ((np.cos(df_join.pred_yaw) - np.cos(df_join.yaw))**2 + (np.sin(df_join.pred_yaw) - np.sin(df_join.yaw))**2)**(1/2)
    df_join['score'] = df_join.mseX+df_join.mse_yaw
    return df_join['score'].mean(), df_join


def make_prediction(pred, stamp_ns):                
    pred_loc = []
    frame_id2predid = {int(row[0]): i for i, row in enumerate(pred)}
    #print(frame_id2predid)
    for t in stamp_ns:                                          
        t = int(t)                                  
        left_pred_id = frame_id2predid[t // FRAME_STEP]
        right_pred_id = left_pred_id + 1
        delta = t % FRAME_STEP
        if right_pred_id >= pred.shape[0]:
            #print(f"Warning {right_pred_id=}, {pred.shape=}")
            bounds = np.concatenate([pred[-2:-1], pred[-1:]], axis=0)
        else:
            bounds = pred[left_pred_id:right_pred_id+1]  
        if len(bounds) != 2:
            raise RuntimeError(f"{bounds}, {t}, {left_pred_id}, {right_pred_id}, {pred.shape}")
        db = bounds[1] - bounds[0]                        
        ddot = db * delta / FRAME_STEP                     
        dot = bounds[0] + ddot                             
        pred_loc.append((t, dot[1], dot[2], dot[3]))
    df_pred = pd.DataFrame(pred_loc, columns=['stamp_ns', 'pred_x', 'pred_y', 'pred_yaw'])
    return df_pred
    

def calculate_ideal_score_quant(d):
    #pred = get_pred_from_score(d)
    pred = pd.read_csv(f'{d}/localization-quant20mc_v3.csv')[['frame_id', 'x', 'y', 'yaw']].values
    with open(f'{d}/quant20mc_v3') as f:
        start_ns=float(f.read())
    loc_df = pd.read_csv(f'{d}/localization.csv') 
    loc_df['stamp_ns'] = loc_df.stamp_ns - start_ns
    df_pred = make_prediction(pred, loc_df.stamp_ns)
    return measure_score(df_pred, loc_df)
    
score, df = calculate_ideal_score_quant('YandexCup2024v2/YaCupTest/0')
print(score)
def calculate_zero_score(d):
    #pred = get_pred_from_score(d)
    loc_df = pd.read_csv(f'{d}/localization.csv') 
    with open(f'{d}/quant20mc_v3') as f:
        start_ns=float(f.read())
    loc_df['stamp_ns'] = loc_df.stamp_ns - start_ns
    df_pred = pd.DataFrame(loc_df[['stamp_ns', 'x', 'y', 'yaw']].values, columns=['stamp_ns', 'pred_x', 'pred_y', 'pred_yaw'])
    return measure_score(df_pred, loc_df)
score, df = calculate_zero_score('YandexCup2024v2/YaCupTest/0')
print(score)

KeyError: "['frame_id'] not in index"

In [ ]:
results = []
for d in tqdm([*[*glob('YandexCup2024v2/YaCupTest/*')][::5], *[*glob('YandexCup2024v2/YaCupTrain/10*')][::5]]):
    zero_score, _ = calculate_zero_score(d)
    quant_score, _ = calculate_ideal_score_quant(d)
    full_score, df = calculate_ideal_score(d)
    #print(quant_score)
    results.append((d, zero_score, quant_score, full_score))
    #print(results[-1])

In [ ]:
results_np = np.asarray([list(map(float, t)) for _, *t in results])
results_np.mean(axis=0)

In [ ]:
results_np.argmax(axis=0)

In [65]:
df = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3', 'K4', 'K5'],
                   'A': ['A0', 'A1', 'A2', 'A3', 'A4', 'A5']})
other = pd.DataFrame({'key': ['K0', 'K1', 'K2'],
                      'B': ['B0', 'B1', 'B2']})
df.set_index('key').join(other.set_index('key'))

,A,B
key,,
K0,A0,B0
K1,A1,B1
K2,A2,B2
K3,A3,NaN
K4,A4,NaN
K5,A5,NaN


In [66]:
other.set_index('key').join(df.set_index('key'))

,B,A
key,,
K0,B0,A0
K1,B1,A1
K2,B2,A2


In [77]:
a = torch.nn.functional.pad(torch.arange(30).view(2,3,5), (0,0, 1, 0), value=-1000)
print(a.shape)
a

torch.Size([2, 4, 5])


tensor([[[-1000, -1000, -1000, -1000, -1000],
         [    0,     1,     2,     3,     4],
         [    5,     6,     7,     8,     9],
         [   10,    11,    12,    13,    14]],

        [[-1000, -1000, -1000, -1000, -1000],
         [   15,    16,    17,    18,    19],
         [   20,    21,    22,    23,    24],
         [   25,    26,    27,    28,    29]]])

In [122]:
data = torch.load('tmp/YaCupTrain__11687.pth')

/tmp/ipykernel_353021/121842413.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('tmp/YaCupTrain__11687.pth')


In [123]:
data.keys()

dict_keys(['batch', 'attention_mask', 'mask', 'inputs_embeds', 'labels_w_mask', 'outputs', 'logits_w_mask', 'loss'])

In [124]:
data['outputs']['loc.pth'][0]

tensor([[ 2.2097e+04, -6.5113e+04,  3.4293e+01,  1.0708e-01, -1.7448e-01,
         -2.8186e+00],
        [ 2.2097e+04, -6.5113e+04,  3.4290e+01, -2.9861e-02,  4.5324e-02,
         -2.7439e+00],
        [ 2.2096e+04, -6.5111e+04,  3.4629e+01,  8.5670e-02, -2.4974e-02,
         -2.5952e+00],
        ...,
        [ 2.2154e+04, -6.5281e+04,  3.1793e+01, -1.6713e+00,  2.4100e+00,
         -4.7205e+00],
        [ 2.2163e+04, -6.5283e+04,  3.6307e+01, -2.5581e+00, -1.8996e+00,
         -4.8193e+00],
        [ 2.2160e+04, -6.5283e+04,  3.5243e+01, -4.0093e+00, -2.4420e-01,
         -1.3853e+00]], device='cuda:0', grad_fn=<SelectBackward0>)

In [125]:
data['outputs']['logits.pth'][0]

tensor([[-7.4201e-01, -2.1288e-01,  2.4525e-01,  9.6820e-02, -1.6709e-01,
          4.4357e-02],
        [-1.0309e+00, -1.6344e-02,  2.4240e-01, -4.0124e-02,  5.2717e-02,
          1.1906e-01],
        [-1.5823e+00,  1.2993e+00,  5.8091e-01,  7.5407e-02, -1.7581e-02,
          2.6778e-01],
        ...,
        [ 5.5628e+01, -1.6816e+02, -2.2544e+00, -1.6816e+00,  2.4174e+00,
         -1.8575e+00],
        [ 6.5307e+01, -1.7079e+02,  2.2589e+00, -2.5684e+00, -1.8922e+00,
         -1.9563e+00],
        [ 6.2129e+01, -1.7051e+02,  1.1957e+00, -4.0196e+00, -2.3681e-01,
          1.4776e+00]], device='cuda:0', grad_fn=<SelectBackward0>)

In [126]:
data['batch'].keys()

dict_keys(['__key__', 'loc.pth', 'control_feats.pth'])

In [127]:
data['labels_w_mask'][0]

tensor([-0.1309, -0.0352,  0.0010,  0.0003, -0.0003,  0.0027], device='cuda:0')

In [128]:
data['logits_w_mask'][0]

tensor([-0.7420, -0.2129,  0.2453,  0.0968, -0.1671,  0.0444], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [129]:
data['batch']['loc.pth'][0]

tensor([[ 2.2098e+04, -6.5113e+04,  3.4048e+01,  1.0263e-02, -7.3928e-03,
         -2.8629e+00],
        [ 2.2098e+04, -6.5113e+04,  3.4049e+01,  1.0582e-02, -7.7387e-03,
         -2.8603e+00],
        [ 2.2098e+04, -6.5113e+04,  3.4051e+01,  1.0162e-02, -7.9580e-03,
         -2.8575e+00],
        ...,
        [ 2.2183e+04, -6.5354e+04,  3.5068e+01,  1.4019e-02,  2.0624e-03,
         -7.9009e-01],
        [ 2.2183e+04, -6.5354e+04,  3.5063e+01,  1.6044e-02,  1.6661e-03,
         -7.8786e-01],
        [-1.0000e+06, -1.0000e+06, -1.0000e+06, -1.0000e+06, -1.0000e+06,
         -1.0000e+06]], device='cuda:0')

In [113]:
data['inputs_embeds'][0]

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -8.1063e-02,
          1.0000e+00,  2.0000e+00],
        [-1.5625e-01, -4.8340e-02,  4.4851e-03,  ..., -8.2834e-02,
          1.0000e+00,  2.0000e+00],
        [-3.1152e-01, -9.6191e-02,  8.5697e-03,  ..., -8.4605e-02,
          1.0000e+00,  2.0000e+00],
        ...,
        [ 2.3125e+02, -7.4085e+02,  1.0253e+01,  ..., -1.7573e-02,
          1.0000e+00,  2.0000e+00],
        [ 2.3135e+02, -7.4114e+02,  1.0262e+01,  ..., -1.7573e-02,
          1.0000e+00,  2.0000e+00],
        [ 2.3144e+02, -7.4142e+02,  1.0270e+01,  ..., -1.7573e-02,
          1.0000e+00,  2.0000e+00]], device='cuda:0')